# Scenarios, Numerical integration

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from scipy import optimize
from scipy import integrate

from tqdm import tqdm_notebook as tqdm
import timeit

### Best response functions & Utilities

In [ ]:
def BR_OPTIMIZE_f(v,rv) :
    """
    Using optimize.minimize, it returns the best response function against the NON-NEGATIVE random variable rv :
    v \mapsto argmax_b (v-b)G(b)
    where G is the cdf of rv
    """
    minus_utility_v = lambda b : -(v-b)*rv.cdf(b)
    opt = optimize.minimize_scalar(minus_utility_v,method="bounded",bracket=(0,1),bounds=(0,1))
    return opt.x

In [ ]:
def array_to_f(x,x_,y_) :
    return y_[min(np.searchsorted(x_,x),len(x_)-1)]

In [ ]:
def array_to_inverse_f(y,x_,y_) :
    return x_[min(np.searchsorted(y_,y),len(y_)-1)]

In [ ]:
def f_to_array(f,x_,) :
    return np.array([f(x) for x in x_])

### Marshall methods

In [ ]:
def marshall1_sub_method(tstar,k1,k2,N=10_000,p=5,correct=True) :
    l1 = 1+1/k2
    l2 = 1+1/k1
    
    output = {}
    output['tstar'] = tstar
    output['N']=N
    output['breakpoint'] = 0
    output['l1'] = l1
    output['l2'] = l2
    output['k1'] = k1
    output['k2'] = k2
    
    delta1_ = np.zeros(N+2)
    delta2_ = np.zeros(N+2)

    a_ = np.zeros(p+1)
    b_ = np.zeros(p+1)
    
    delta1_[N+1] = 1/tstar
    delta2_[N+1] = 1/tstar
    
    for j in np.arange(N+1,0,-1) : #j=N+1,...,1
        tj = tstar*j/(N+1)
                    
        #values at tj, equation (17)
        a_[0] = delta1_[j] 
        b_[0] = delta2_[j]
        
        ## updating the Taylors approximations astar_,bstar_ equations (19, 20)
        
        for l in range(p) :
            sum1 = np.sum([i*b_[l+1-i]*( a_[i-1]+tj*a_[i]) for i in range(1,l+1)])
            sum2 = np.sum([i*a_[l+1-i]*( b_[i-1]+tj*b_[i]) for i in range(1,l+1)])
    
            a_[l+1] = 1/((l+1)*(b_[0]-1)*tj) * ( (1/k1-(l+1)*(b_[0]-1))*a_[l]-sum1 )
            b_[l+1] = 1/((l+1)*(a_[0]-1)*tj) * ( (1/k2-(l+1)*(a_[0]-1))*b_[l]-sum2 )
        
        tjm1 = tstar*(j-1)/(N+1)
            
        delta1_[j-1] = np.polynomial.polynomial.polyval(tjm1-tj,a_) #updating at tj-1
        delta2_[j-1] = np.polynomial.polynomial.polyval(tjm1-tj,b_)
        
        if(output['breakpoint'] == 0 and ((delta1_[j-1]-l1)**2+(delta2_[j-1]-l2)**2 > ((delta1_[j]-l1)**2+(delta2_[j]-l2)**2))) :
            output['breakpoint'] = j
            if(correct) :
                break
    
    output['eps_star'] = np.sqrt(((delta1_[output['breakpoint']]-l1)**2+(delta2_[output['breakpoint']]-l2)**2))
    #output['precision'] = np.min(((delta1_-l1)**2+(delta2_-l2)**2))
    if(correct) :
        ind = output['breakpoint']
        delta1_[:ind+1] = np.linspace(l1,delta1_[ind],ind+1)
        delta2_[:ind+1] = np.linspace(l2,delta2_[ind],ind+1)
        
        #patch_ = 1/(np.linspace(0,out['tstar'],out['N']+2)[1:])
        #delta1_[1:] = np.where(delta1_[1:] > patch_, patch_,delta1_[1:])
        #delta2_[1:] = np.where(delta2_[1:] > patch_, patch_,delta2_[1:])
        
    output['delta1_']=delta1_
    output['delta2_']=delta2_
    output['values1_']=np.linspace(0,tstar,N+2)*delta1_
    output['values2_']=np.linspace(0,tstar,N+2)*delta2_
    return output

In [ ]:
def marshall1_close_form_boundary(k1,k2,N=10_000,p=5,correct=True) :
    tstar = k1/(k1+1)
    
    if(k1!=k2) :
        tstar = 1-np.power(1+k1,k2/(k1-k2))/np.power(1+k2,k1/(k1-k2))*np.power((k2*(1+k1))/(k1*(1+k2)),k1*k2/(k1-k2))
        
    return marshall1_sub_method(tstar,k1,k2,N=N,p=p,correct=correct)

In [ ]:
def marshall2_sub_method(tstar,k1,k2,N=10_000,p=5,correct=True) :
    delta1_ = np.zeros(N+2)
    delta2_ = np.zeros(N+2)
    
    l1 = 1+1/k2
    l2 = 1+1/(k1+k2-1)
    
    output = {}
    output['tstar'] = tstar
    output['N']=N
    output['breakpoint'] = 0
    output['l1'] = l1
    output['l2'] = l2
    
    a_ = np.zeros(p+1)
    b_ = np.zeros(p+1)
    c_ = np.zeros(p+1)
    alpha_ = np.zeros(p+1)
    beta_ = np.zeros(p+1)
    gamma_ = np.zeros(p+1)
    
    theta = (k2-1)/k1
    
    delta1_[N+1] = 1/tstar
    delta2_[N+1] = 1/tstar
    
    for j in np.arange(N+1,0,-1) : #j=N+1,...,1
        tj = tstar*j/(N+1)
                    
        #values at tj, equations (17 (modified, see 26), 36, 37)
        b_[0] = delta2_[j]
        
        #c_[0] = (delta1_[j]**(k1/(k1+k2-1)))*(delta2_[j]**((k2-1)/(k1+k2-1)))
        #a_[0] = c_[0]*(c_[0]/b_[0])**theta
        
        a_[0] = delta1_[j]
        c_[0] = (a_[0]**(k1/(k1+k2-1)))*(b_[0]**((k2-1)/(k1+k2-1)))
        
        alpha_[0] = a_[0]*c_[0]
        beta_[0] = b_[0]*c_[0]
        gamma_[0] = a_[0]*b_[0]
        
        ## updating the Taylors approximations equations (16, 33, 34, 35)
        for l in range(p) :
            sum_b = np.sum([i*a_[l+1-i]*( b_[i-1]+tj*b_[i]) for i in range(1,l+1)])    
            b_[l+1] = -b_[l]/tj + 1/((l+1)*(a_[0]-1)*tj) * ( (1/k2)*b_[l]-sum_b )
        
            sum_c = np.sum([i*b_[l+1-i]*( c_[i-1]+tj*c_[i]) for i in range(1,l+1)])
            c_[l+1] = -c_[l]/tj + 1/((l+1)*(b_[0]-1)*tj) * ( (1/(k1+k2-1))*c_[l]-sum_c )
            
            A = np.sum([i*c_[i]*gamma_[l+1-i]-a_[i]*beta_[l+1-i] for i in range(1,l+1)]) 
            B = theta*np.sum([i*c_[i]*gamma_[l+1-i]-b_[i]*alpha_[l+1-i] for i in range(1,l+1)])
            a_[l+1] = c_[l+1]*alpha_[0]/beta_[0]+theta*(c_[l+1]*alpha_[0]/beta_[0]-b_[l+1]*alpha_[0]/beta_[0])+1/((l+1)*beta_[0]) * (A+B)
            
            alpha_[l+1] = np.sum([a_[i]*c_[l+1-i] for i in range(0,l+2)]) 
            beta_[l+1] = np.sum([b_[i]*c_[l+1-i] for i in range(0,l+2)]) 
            gamma_[l+1] = np.sum([a_[i]*b_[l+1-i] for i in range(0,l+2)]) 
        
        tjm1 = tstar*(j-1)/(N+1)
        
        delta1_[j-1] = np.polynomial.polynomial.polyval(tjm1-tj,a_) #updating at tj-1
        delta2_[j-1] = np.polynomial.polynomial.polyval(tjm1-tj,b_)
        
        if(output['breakpoint'] == 0 and ((delta1_[j-1]-l1)**2+(delta2_[j-1]-l2)**2 > ((delta1_[j]-l1)**2+(delta2_[j]-l2)**2))) :
            output['breakpoint'] = j
            if(correct) :
                break
    
    output['eps_star'] = np.sqrt(((delta1_[output['breakpoint']]-l1)**2+(delta2_[output['breakpoint']]-l2)**2))
    #output['precision'] = np.min(((delta1_-l1)**2+(delta2_-l2)**2))
    if(correct) :
        ind = output['breakpoint']
        delta1_[:ind+1] = np.linspace(l1,delta1_[ind],ind+1)
        delta2_[:ind+1] = np.linspace(l2,delta2_[ind],ind+1)
        
        #patch_ = 1/(np.linspace(0,out['tstar'],out['N']+2)[1:])
        #delta1_[1:] = np.where(delta1_[1:] > patch_, patch_,delta1_[1:])
        #delta2_[1:] = np.where(delta2_[1:] > patch_, patch_,delta2_[1:])
        
    output['delta1_']=delta1_
    output['delta2_']=delta2_
    output['values1_']=np.linspace(0,tstar,N+2)*delta1_
    output['values2_']=np.linspace(0,tstar,N+2)*delta2_
    return output

In [ ]:
def marshall2_iterative_find_boundary_CUSTOM(k1,k2,eps=10**-5,nb_eval_max=500,N=10_000,p=5,show_msg=False) :
    l1 = 1+1/k2
    l2 = 1+1/(k1+k2-1)
    
    a = 1/l1
    b = 1/l2
    
    tau = (np.sqrt(5)-1)/2
    
    x_1 = a + (1-tau)*(b-a)
    f_1 = marshall2_sub_method(x_1,k1,k2,N,p,correct=True)
    
    x_2 = a + tau*(b-a)
    f_2 = marshall2_sub_method(x_2,k1,k2,N,p,correct=True)
    
    i=2
    while(f_1['eps_star']>eps and f_2['eps_star']>eps and i < nb_eval_max) :
        if(f_1['eps_star'] > f_2['eps_star']) :
            a = x_1
            x_1 = x_2
            f_1 = f_2
            x_2 = a + tau*(b-a)
            f_2 = marshall2_sub_method(x_2,k1,k2,N,p,correct=True)
        else :
            b = x_2
            x_2 = x_1
            f_2 = f_1
            x_1 = a + (1-tau)*(b-a)
            f_1 = marshall2_sub_method(x_1,k1,k2,N,p,correct=True)
        i+=1
    
    if(show_msg) :
        print("Nb of eval :",i)
        print("eps_star :",min(f_1['eps_star'],f_2['eps_star']))
        if(i==nb_eval_max) :
            print("Maximum number of evaluations reached")
            
    if(f_1['eps_star']<f_2['eps_star']) :
        return f_1
    else :
        return f_2

### Scenarios

In [ ]:
def scenario1(k,n) :
    """
    The marketplace is bidding second highest report, a countercoallition is playing strategically
    """
    N=10_000
    v_ = np.linspace(0,1,N+1)
    
    RV = stats.beta(a=k-1,b=2)# F^k_(2)
    br_ = f_to_array(lambda v:BR_OPTIMIZE_f(v,RV), v_)
    
    beta_out = lambda v:array_to_f(v,v_,br_)
    beta_out_inverse = lambda b:array_to_inverse_f(b,v_,br_)
        
    integrand_in = lambda v2,v1: (v1-v2)*((beta_out_inverse(v2))**(n-k))*k*(k-1)*1*1*v2**(k-2)
    integrand_out = lambda v : (v-beta_out(v))*RV.cdf(beta_out(v))*((n-k)*v**(n-k-1))
    integrand_auct = lambda v: v*(beta_out_inverse(v))**(n-k)*RV.pdf(v)+beta_out(v)*RV.cdf(beta_out(v))*(n-k)*(v**(n-k-1))
    
    output = {}
    
    # Outer integral : x=v1
    # Inner integral : y=v2
    output['in'] = integrate.dblquad(integrand_in, 0, 1, lambda x:0, lambda x:x)[0]/k
    output['out'] = integrate.quad(integrand_out,0,1)[0]/(n-k)
    output['auct'] = integrate.quad(integrand_auct,0,1)[0]
    
    return output

In [ ]:
def scenario2(k,n,N=10_000) :
    """
    The marketplace is playing against a coutercoallition, both plays strategically the NE
    """
    
    out = marshall1_close_form_boundary(k,n-k,N=N)
    b_ = np.linspace(0,out['tstar'],N+2)
    
    
    v_in_f = lambda b:array_to_f(b,b_,out['values1_'])
    v_out_f = lambda b:array_to_f(b,b_,out['values2_'])
    b_in_f = lambda v:array_to_inverse_f(v,b_,out['values1_'])
    b_out_f = lambda v:array_to_inverse_f(v,b_,out['values2_'])
    
    
    integrand_in = lambda v : (v-b_in_f(v))*(v_out_f(b_in_f(v))**(n-k))*k*v**(k-1)
    integrand_out = lambda v : (v-b_out_f(v))*((v_in_f(b_out_f(v)))**k)*((n-k)*v**(n-k-1))
    integrand_auct = lambda v : b_in_f(v)*((v_out_f(b_in_f(v)))**(n-k))*k*v**(k-1) + b_out_f(v)*((v_in_f(b_out_f(v)))**k)*(n-k)*v**(n-k-1)
    
    output = {}
    
    output['in'] = integrate.quad(integrand_in, 0, 1)[0]/k
    output['out'] = integrate.quad(integrand_out,0,1)[0]/(n-k)
    output['auct'] = integrate.quad(integrand_auct,0,1)[0]
    
    return output

In [ ]:
def scenario3(k,n,N=10_000) :
    """
    The marketplace is playing against a coutercoallition, both plays strategically the NE
    """
    
    out = marshall2_iterative_find_boundary_CUSTOM(k,n-k,N=N,eps=10**-3)
    b_ = np.linspace(0,out['tstar'],N+2)
    
    
    v_in_f = lambda b:array_to_f(b,b_,out['values1_'])
    v_out_f = lambda b:array_to_f(b,b_,out['values2_'])
    b_in_f = lambda v:array_to_inverse_f(v,b_,out['values1_'])
    b_out_f = lambda v:array_to_inverse_f(v,b_,out['values2_'])
    
    
    integrand_in = lambda v : (v-b_in_f(v))*(v_out_f(b_in_f(v))**(n-k))*k*v**(k-1)
    integrand_out = lambda v : (v-b_out_f(v))*((v_in_f(b_out_f(v)))**k)*((n-k)*v**(n-k-1))
    integrand_auct = lambda v : b_in_f(v)*((v_out_f(b_in_f(v)))**(n-k))*k*v**(k-1) + b_out_f(v)*((v_in_f(b_out_f(v)))**k)*(n-k)*v**(n-k-1)
    
    output = {}
    
    output['in'] = integrate.quad(integrand_in, 0, 1)[0]/k
    output['out'] = integrate.quad(integrand_out,0,1)[0]/(n-k)
    output['auct'] = integrate.quad(integrand_auct,0,1)[0]
    
    return output

## Marshall1994 tests, Table III

In [ ]:
n = 5
k_ = [1,2,3,4]

res_ = {'auct':[],'in':[],'out':[]}
for k in tqdm(k_) :
    output = scenario2(k,n)
    
    res_['in'].append(output['in'])
    res_['out'].append(output['out'])
    res_['auct'].append(output['auct'])
    
print("Coallition vs Coallition")
for i in range(len(k_)):
    print("k1 {} | k2 {} | auct. revenue {:0.4f} | k1 member surplus {:0.4f} | k2 member surplus {:0.4f}".format(k_[i],n-k_[i],res_['auct'][i],res_['in'][i],res_['out'][i]))

In [ ]:
n = 5
k_ = [1,2,3,4]

res_ = {'auct':[],'in':[],'out':[]}
for k in tqdm(k_) :
    output = scenario3(k,n)
    
    res_['in'].append(output['in'])
    res_['out'].append(output['out'])
    res_['auct'].append(output['auct'])
    
print("Coallition vs Individuals")
for i in range(len(k_)):
    print("k1 {} | k2 {} | auct. revenue {:0.4f} | k1 member surplus {:0.4f} | k2 member surplus {:0.4f}".format(k_[i],n-k_[i],res_['auct'][i],res_['in'][i],res_['out'][i]))

### Custom tests

In [ ]:
n = 50
k_ = np.arange(2,n)

scenarios_ = [{'auct':[],'in':[],'out':[]} for _ in range(4)]

for k in tqdm(k_) :    
    output = scenario1(k,n)
    scenarios_[1]['auct'].append(output['auct'])
    scenarios_[1]['in'].append(output['in'])
    scenarios_[1]['out'].append(output['out'])
    
    output = scenario2(k,n)
    scenarios_[2]['auct'].append(output['auct'])
    scenarios_[2]['in'].append(output['in'])
    scenarios_[2]['out'].append(output['out'])
        
    output = scenario3(k,n)
    scenarios_[3]['auct'].append(output['auct'])
    scenarios_[3]['in'].append(output['in'])
    scenarios_[3]['out'].append(output['out'])

In [ ]:
plt.figure(figsize=(15,10))
plt.xlabel("k")

#plt.semilogy(k_,0*k_+1/(n*(n+1)),'-',color='gray',label="0 : No marketplace (NE) [in/out]")
#plt.semilogy(k_,scenarios_[1]['in'],'-o',color='red',label=r"1 : $r_{(2)}$ marketplace vs countercoallition [in]")
#plt.semilogy(k_,scenarios_[2]['in'],'-o',color='blue',label=r"2 : marketplace vs countercoallition (NE) [in]")
plt.semilogy(k_,scenarios_[3]['in'],'-o',color='green',label=r"3 : marketplace vs individuals (NE) [in]")

#plt.semilogy(k_,scenarios_[1]['out'],'-^',color='red',label=r"1 : $r_{(2)}$ marketplace vs countercoallition [out]")
#plt.semilogy(k_,scenarios_[2]['out'],'-^',color='blue',label=r"2 : marketplace vs countercoallition (NE) [out]")
plt.semilogy(k_,scenarios_[3]['out'],'-^',color='green',label=r"3 : marketplace vs individuals (NE) [out]")

#plt.semilogy(k_,0*k_+(n-1)/(n+1),'-',color='gray',label="0 : No marketplace (NE) [auct]")

#plt.semilogy(k_,scenarios_[1]['auct'],'-+',color='red',label=r"1 : $r_{(2)}$ marketplace vs countercoallition [auct]")
#plt.semilogy(k_,scenarios_[2]['auct'],'-+',color='blue',label=r"2 : marketplace vs countercoallition (NE) [auct]")
#plt.semilogy(k_,scenarios_[3]['auct'],'-+',color='green',label=r"3 : marketplace vs individuals (NE) [auct]")
plt.xlim((1,15))
plt.ylim((0,0.5*10**-3))
plt.legend()
plt.show()